In [102]:
# Packages/functions importation
import igraph as ig
import random
from math import floor
import time

## Percolation threshold depending on car density

### Generating the random parking

In [89]:
#Auxiliary functions

def generate_prufer_sequence(n):
    """
    Generate the Prufer sequence for a tree with n vertices.
    
    Arguments:
    n -- (int) size of the tree/number of vertices 
    
    Returns:
    prufer -- (list) random prufer sequences characterising the tree
    """
    
    prufer = [random.randint(0,n-1) for i in range(n-2)] # we draw n-2 random vertices id (repetition is possible with the Prufer sequence)
    return prufer



def build_random_tree(prufer):
    """
    Build the tree corresponding to the Prufer sequence
    
    Arguments:
    prufer -- (list) random prufer sequences characterising the tree
    
    Returns:
    g -- (Graph) tree with n vertices generated by the Prufer sequence
    """
    
    n = len(prufer) + 2 # deduce the number of vertices of the graph based on the Prufer sequence
    
    # Initialisations
    g = ig.Graph() # create an empty graph
    g.add_vertices(n) # add the n isolated vertices to the graph
    vertex_list = range(n) # we save the list of vertices in a variable for performance reasons
    
    # Determine the vertices degree to have in the final tree
    degree = [1 for i in range(n)] # g being a tree, the degree of each vertex is necessarily at least 1
    for vertex in prufer:
        degree[vertex] +=1 # add 1 to the vertex degree whenever encountered in the Prufer sequence
        
    # Add the edges
    for vertex_i in prufer:
        for vertex_j in vertex_list:
            if degree[vertex_j]==1:
                g.add_edge(vertex_i, vertex_j)
                degree[vertex_i]-=1
                degree[vertex_j]-=1
                break
                
    # Add the last edge
    u=0
    v=0
    for vertex in vertex_list:
        if degree[vertex]==1:
            if u == 0:
                u = vertex
            else:
                v=vertex
                break
    g.add_edge(u,v)
    degree[u]-=1
    degree[v]-=1
    
    return g
    


In [93]:
# Generate the parking

n = 5000 # number of parking spots
parking = build_random_tree(generate_prufer_sequence(n))
root = random.randint(0,n-1) # choose the root randomly
print("Root: ",root)

# Plot parameters
layout = parking.layout_reingold_tilford(root=root)
#vertex_label=range(n)

ig.summary(parking)
ig.plot(parking, 
        layout=layout, 
        vertex_size=2)

Root:  3557
IGRAPH U--- 5000 4999 -- 


### Adding the cars

In [72]:
# Auxiliary functions for introduction of cars

def explore_levels(g, root):
    """
    Creates a matrix sorting the vertices by depth from the root
    
    Arguments:
    g -- (Graph) graph
    root -- (int) root of the tree 
    
    Returns:
    depth_to_vertices -- (matrix) such that depth_to_vertices[d] gives all the vertices at depth d from the root
    vertices_to_depth -- (list) such that vertices_to_depth[v] gives the depth of vector v
    """
    
    depth = 0
    depth_to_vertices=[]
    depth_to_vertices.append([root])
    
    vertices_to_depth=[-1 for i in range(len(g.vs))]
    vertices_to_depth[root]=0
    
    while True:
        depth+=1
        levels_depth=[]
        
        for vertex in depth_to_vertices[depth -1]:
            neighbours = g.neighbors(vertex)
            
            for neighbour in neighbours:
                
                if depth == 1:
                    levels_depth.append(neighbour)
                    vertices_to_depth[neighbour]=depth
                else:
                    if neighbour not in depth_to_vertices[depth-2]:

                        levels_depth.append(neighbour) #make sure not to add previous vertices
                        vertices_to_depth[neighbour]=depth
            
        if levels_depth == []: #stop the loop when no more depth to explore
            break
        
        depth_to_vertices.append(levels_depth)
        
    return depth_to_vertices, vertices_to_depth



def move_car(parking, initial_position, occupancy, unparked,vertices_to_depth):
    """Moves a car and update the state of the parking spot
    
    Arguments:
    parking -- (Graph) random parking generated
    initial_position -- (int) id of the random vertex on which the car appears
    occupancy -- (list) vector of Booleans such that occupancy[v]= True if the spot is already taken
    unparked -- (int) number of cars that were unable to park
    vertices_to_depth -- (list) such that vertices_to_depth[v] gives the depth of vector v
    
    Returns:
    occupancy -- updated version after the car has moved
    unparked -- updated version after the car has moved 
    """
    
    car_position = initial_position
    car_depth = vertices_to_depth[initial_position]
        
    while car_depth > 0 and occupancy[car_position]:
        
        neighbours = parking.neighbors(car_position)
        for neighbour in neighbours:
            if vertices_to_depth[neighbour]==car_depth-1:
                car_position = neighbour # the car moves to its neighbour closer to the root
        
        car_depth = vertices_to_depth[car_position]
    
    if not occupancy[car_position]: # the car has found an empty parking spot and parks there
        occupancy[car_position] = True 
        
    else:  # the car has made its way to the root but hasn't found an empty spot to park on
        unparked +=1
    
    return occupancy,unparked
    

In [106]:
# Putting things together - 1 experiment

# Setting the model parameters
n = 5000 # number of parking spots

parking = build_random_tree(generate_prufer_sequence(n)) # build parking
root = random.randint(0,n-1) # choose the root randomly
alpha = 0.3 # density of cars
m = floor(alpha*n) # number of cars

# Initialisations

depth_to_vertices,vertices_to_depth=explore_levels(parking,root)
occupancy = [False for i in range(n)] # Occupancy vector of the parking spots, originally all spots are free
unparked=0

for car in range(m):
    initial_position=random.randint(0,n-1)
    occupancy, unparked= move_car(parking,initial_position,occupancy,unparked,vertices_to_depth)
    
print("Nb of unparked cars: ",unparked)

Nb of unparked cars:  0


In [83]:
# Repeating experiments
def simulate_parking(n,alpha):
    """
    Runs the parking model with n parking spots and a car density
    
    n -- (int) number of parking spots
    alpha -- (int) car density
    
    Returns:
    unparked -- (int)number of parked cars
    """
    
    parking = build_random_tree(generate_prufer_sequence(n))
    root = random.randint(0,n-1) # choose the root randomly
    m = floor(alpha*n) # number of cars

    # Initialisations
    depth_to_vertices,vertices_to_depth=explore_levels(parking,root)
    occupancy = [False for i in range(n)] # Occupancy vector of the parking spots, originally all spots are free
    unparked=0

    for car in range(m):
        initial_position=random.randint(0,n-1)
        occupancy, unparked= move_car(parking,initial_position,occupancy,unparked,vertices_to_depth)

    return unparked

In [107]:
# We run the simulation several times and compare to theoretical results

# SIMULATION PARAMETERS
N_exp = 100 # number of times we run the simulation
n = 5000 # size of the parking
alpha = 0.6# car density


all_cars_parked_count = 0 # we keep track of the number of times the event "all cars can park" is realised

for t in range(N_exp):
    unparked = simulate_parking(n,alpha)
    if unparked==0:
        all_cars_parked_count+=1
    
print("Results of the simulation: ",all_cars_parked_count/N_exp)


# Compare with expected probability

if alpha<=1/2:
    expected_proba = ((1-2*alpha)**(1/2))/(1-alpha)    
else:
    expected_proba = 0

print("Expected probability: ", expected_proba)

Results of the simulation:  0.0
Expected probability:  0


In [ ]:
# Generating figures

N_exp = 1000

alphas = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7] # list of alphas to test
n_list = [50, 500, 1000, 2000, 5000, 10000] # list of parking sizes to test



## Introducing car removal